# Read Data

In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.image as img


import numpy as np
import pandas as pd
import json
import matplotlib.image as img
from keras.preprocessing.image import ImageDataGenerator, Iterator, flip_axis
from keras.layers.core import Dense, Flatten, Activation, SpatialDropout2D, Lambda, Dropout
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.objectives import mse
from sklearn.model_selection import train_test_split

import numpy as np
from scipy.misc import imread, imresize

import matplotlib.pyplot as plt

%matplotlib inline

# 1. Read Data from Driving Log
driving_log = pd.read_csv('driving_log.csv', index_col=False)
driving_log.columns = ['center_imgpath', 'left_imgpath', 'right_imgpath', 'angle', 'throttle', 'break', 'speed']

# 2. Prepare Data for Generator
X_train_path, y_train = [], []
for index, row in driving_log.iterrows():
    # Include center image and steering angle.
    X_train_path.append(row['center_imgpath'])
    y_train.append(row['angle'])
    
    
    C = row['angle']
    L = C + 0.07
    R = C - 0.07
        
    X_train_path.append(row['left_imgpath'].strip())
    y_train.append(L)
        
    X_train_path.append(row['right_imgpath'].strip())
    y_train.append(R)
    
  
X_train_path, y_train = np.array(X_train_path), np.array(y_train)

Using TensorFlow backend.


In [2]:
def read_imgs(img_paths):
    """
    read images from file path.
    """        
    imgs = np.empty([len(img_paths), 160, 320, 3])

    for i, path in enumerate(img_paths):
        imgs[i] = imread(path)

    return imgs

def resize(X):
    import tensorflow
    return tensorflow.image.resize_images(X, (40, 160))

def gen_batches(imgs, angles, batch_size):
    """
    Generates random batches of the input data.
    :imgs: The input image file paths
    :angles: The steering angles associated with each image.
    :batch_size: The size of each minibatch.
    :yeilds A tuple (images, angles), where both images and angles have batch_size elements.
    """
    num_imgs = len(imgs)

    while True:
        indices = np.random.choice(num_imgs, batch_size)
        batch_imgs_raw, angles_raw = read_imgs(imgs[indices]), angles[indices].astype(float)
        yield batch_imgs_raw, angles_raw
        
def get_model():
    
    model = Sequential([
        # Preprocess
        # Resize images to improve performance
        # Normalize to keep weight values small with zero mean, improving numerical stability.
        Lambda(resize, input_shape=(160, 320, 3)),
        BatchNormalization(axis=1),
        # Conv 5x5
        Convolution2D(24, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 5x5
        Convolution2D(36, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 5x5
        Convolution2D(48, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Flatten
        Flatten(),
        # Fully Connected
        Dense(100, activation='elu', W_regularizer=l2(1e-6)),
        Dense(50, activation='elu', W_regularizer=l2(1e-6)),
        Dense(10, activation='elu', W_regularizer=l2(1e-6)),
        Dense(1)
    ])
    
    return model

def save_model(model):
    with open('model.json', 'w') as outfile:
        json.dump(model.to_json(), outfile)
    model.save_weights('model.h5')

    print ('Model saved!')
    
    return Truet tensorflow
    return tensorflow.image.resize_images(X, (40, 160))

def gen_batches(imgs, angles, batch_size):
    """
    Generates random batches of the input data.
    :imgs: The input image file paths
    :angles: The steering angles associated with each image.
    :batch_size: The size of each minibatch.
    :yeilds A tuple (images, angles), where both images and angles have batch_size elements.
    """
    num_imgs = len(imgs)

    while True:
        indices = np.random.choice(num_imgs, batch_size)
        batch_imgs_raw, angles_raw = read_imgs(imgs[indices]), angles[indices].astype(float)
        yield batch_imgs_raw, angles_raw
        
def get_model():
    
    model = Sequential([
        # Preprocess
        # Resize images to improve performance
        # Normalize to keep weight values small with zero mean, improving numerical stability.
        Lambda(resize, input_shape=(160, 320, 3)),
        BatchNormalization(axis=1),
        # Conv 5x5
        Convolution2D(24, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 5x5
        Convolution2D(36, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 5x5
        Convolution2D(48, 5, 5, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Flatten
        Flatten(),
        # Fully Connected
        Dense(100, activation='elu', W_regularizer=l2(1e-6)),
        Dense(50, activation='elu', W_regularizer=l2(1e-6)),
        Dense(10, activation='elu', W_regularizer=l2(1e-6)),
        Dense(1)
    ])
    
    return model

def save_model(model):
    with open('model.json', 'w') as outfile:
        json.dump(model.to_json(), outfile)
    model.save_weights('model.h5')

    print ('Model saved!')
    
    return True


In [3]:
import time

start = time.clock()

model = get_model()
model.compile(optimizer='adam', loss = 'mse')

X_train, X_val, y_train, y_val = train_test_split(X_train_path, y_train)

nb_val_samples = len(y_val)
gen = gen_batches(X_train, y_train, 128)

model.fit_generator(gen,
    samples_per_epoch=len(y_train),
    validation_data=gen_batches(X_val,y_val, 128),
    nb_val_samples=nb_val_samples,
    nb_epoch=5
)

# Save Model/Weights
save_model(model)

end = time.clock()
print ("Training completed in {:2f}s".format(end - start))


Epoch 1/5
6016/6027 [============================>.] - ETA: 0s - loss: 0.1851

/home/carnd/anaconda3/envs/carnd/lib/python3.5/site-packages/keras/engine/training.py:1528: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6144/6027 [==============================] - 39s - loss: 0.1820 - val_loss: 0.0189
Epoch 2/5
6144/6027 [==============================] - 35s - loss: 0.0187 - val_loss: 0.0138
Epoch 3/5
6144/6027 [==============================] - 35s - loss: 0.0157 - val_loss: 0.0166
Epoch 4/5
6144/6027 [==============================] - 36s - loss: 0.0150 - val_loss: 0.0132
Epoch 5/5
6144/6027 [==============================] - 35s - loss: 0.0146 - val_loss: 0.0120
Model saved!
Training completed in 189.754389s


# Old Code

In [ ]:
test = X_train_path[0:5]

test2 = []
for line in test:
    test2.append(line.split('IMG',1)[-1])